In [39]:
import requests
import json
import random
import datetime

In [54]:
def get_player_ids_by_country(country):
    '''use the country's 2-character ISO 3166 code (capitalized) to specify which country you want data 
    for example: United States = US'''
    
    url = f'https://api.chess.com/pub/country/{country}/players'
    response = requests.get(url)
    
    #check if the request worked
    assert response.status_code == 200 
    
    #grab pgn text
    names = str(response.content).split(',')
    
    return [name[1:-2] for name in names][1:]

def date_joined(username, datetype='epoch'):
    '''returns the date the user joined in epoch time by default, human readable date is returned if datetype=readable'''
    
    response = requests.get(f'https://api.chess.com/pub/player/{username}')
    profile = json.loads(response.content.decode('utf-8'))
    try:
        if datetype == 'epoch':
            return profile['joined']
        elif datetype == 'readable':
            return datetime.datetime.fromtimestamp(profile['joined']).strftime('%c')
    except:
        return 0

def blitz_games_won(username):
    response = requests.get(f'https://api.chess.com/pub/player/{username}/stats')
    stats = json.loads(response.content.decode('utf-8'))
    try:
        return stats['chess_blitz']['record']['win']
    except:
        return 0

def list_of_players(country='US', time=1451606400, num_of_players=100, games_won=2):  
    '''use the country's 2-character ISO 3166 code (capitalized) to specify which country you want data
    example: United States = US
    country = player country of origin, default is United States
    time = get players who joined after this time (epochtime) defualt is jan, 1 2016
    num_of_players = number of player ids to retrieve 
    games_won = get players who have won more than a certain amount of games won in blitz, default = 2'''
    
    #random pick from list of usernames
    player_list = random.sample(get_player_ids_by_country(country), 30000)

    players = []

    for player in player_list:
        if len(players) == num_of_players:
            break 
        elif (date_joined(player) > time) and (blitz_games_won(player) >= games_won):
            players.append(player)
    return players

# Important Endpoints from Chess.com

## Player Profile
### Description: Get additional details about a player in a game. 
URL pattern: https://api.chess.com/pub/player/{username} 

{
  "@id": "URL", // the location of this profile (always self-referencing)
  
  "url": "URL", // the chess.com user's profile page (the username is displayed with the original letter case)
  
  "username": "string", // the username of this player
  
  "player_id": 41, // the non-changing Chess.com ID of this player
  
  "title": "string", // (optional) abbreviation of chess title, if any
  
  "status": "string", // account status: closed, closed:fair_play_violations, basic, premium, mod, staff
  
  "name": "string", // (optional) the personal first and last name
  
  "avatar": "URL", // (optional) URL of a 200x200 image
  
  "location": "string", // (optional) the city or location
  
  "country": "URL", // API location of this player's country's profile
  
  "joined": 1178556600, // timestamp of registration on Chess.com
  
  "last_online": 1500661803, // timestamp of the most recent login
  
  "followers": 17 // the number of players tracking this player's activity
}


## Player Stats
### Description: Get ratings, win/loss, and other stats about a player's game play. 
URL pattern: https://api.chess.com/pub/player/{username}/stats

"chess_daily": {
/* stats object for games of rules "chess" and "daily" time-class */


"chess960_daily": {
/* stats object for games of rules "chess960" and "daily" time-class */


"chess_blitz": {
/* stats object for games of rules "chess" and "blitz" time-class */


### In each dictionary:

{
  "last": { // the current stats
  
    "date": 1509709165, // timestamp of the last rated game finished
    
    "rating": 1642, // most-recent rating
    
    "rd": 58 // the Glicko "RD" value used to calculate ratings changes
    
  },
  "best": { // the best rating achieved by a win
  
    "date": 1256228875, // timestamp of the best-win game
    
    "rating": 2065, // highest rating achieved
    
    "game": "URL" // URL of the best-win game
    
  },
  "record": { // summary of all games played
  
    "win": 177,  // number of games won
    
    "loss": 124, // number of games lost
    
    "draw": 21,  // number of games drawn
    
    "time_per_move": 18799, // integer number of seconds per average move
    
    "timeout_percent": 9.99 // timeout percentage in the last 90 days
    
    
  },
  "tournament": { // summary of tournaments participated in
  
    "count": 20,   // number of tournaments joined
    
    "withdraw": 1, // number of tournaments withdrawn from
    
    "points": 39,  // total number of points earned in tournaments
    
    "highest_finish": 1 // best tournament place
  }
}

## List of Monthly Archives
### Description: Array of monthly archives available for this player. 
URL pattern: https://api.chess.com/pub/player/{username}/games/archives


  "archives":
    /* array of URLs for monthly archives in ascending chronological order */

## Complete Monthly Archives
### Description: Array of Live and Daily Chess games that a player has finished. 
URL pattern: https://api.chess.com/pub/player/{username}/games/{YYYY}/{MM}

"YYYY" is the four digit year of the game-end

"MM" is the two-digit month

Data Format, each Game:

{
  "white": { // details of the white-piece player:
  
    "username": "string", // the username
    
    "rating": 1492, // the player's rating at the start of the game
    
    "result": "string", // see "Game results codes" section
    
    "@id": "string", // URL of this player's profile
    
  },
  "black": { // details of the black-piece player:
  
    "username": "string", // the username
    
    "rating": 1942, // the player's rating at the start of the game
    
    "result": "string", // see "Game results codes" section
    
    "@id": "string", // URL of this player's profile
    
  },
  "url": "string", // URL of this game
  
  "fen": "string", // final FEN
  
  "pgn": "string", // final PGN
  
  "start_time": 1254438881, // timestamp of the game start (Daily Chess only)
  
  "end_time": 1254670734, // timestamp of the game end
  
  "time_control": "string", // PGN-compliant time control
  
  "rules": "string", // game variant information (e.g., "chess960")
  
  "eco": "string", //URL pointing to ECO opening (if available),
  
  "tournament": "string", //URL pointing to tournament (if available),  
  
  "match": "string", //URL pointing to team match (if available)  
  
}

In [9]:
player_list = random.sample(get_player_ids_by_country('US'), 30000)

In [32]:
players = []

for player in player_list:
    if len(players) == num_of_players:
        break 
    elif (date_joined(player) > t) and (blitz_games_won(player) > 2):
        players.append(player)
return players

strickj
rashid0
bryg
yellowfupa
mhmmders
ksali
matthewdalton
jo2e
mlamarc
blueduck123
geldon12345
stephen47
rob311
beegal
student808
jorgeaa
kev_98
topat
q7q
eyezek
mhyderali
dbotts5
raximo
aceshi
jimmyro
djd1234
shambhavisri
noor197
sergenian
rod12
dustydust
testud
masterdron
islandof
kantola
excalibir
pidler
johnlong33
krabd
fastenf
badger6
kingjiv
jerronisback
bigshot81
sivasagar
magdy22
zeban
hash777
mn1chess
kelas111
jeanrus
piacentifrank
mrmrs
jharris1
the_chosen_one10
frozenfang
glerps
sd101
trainrec
bagobo
des17
omlu
crnrfght
chrondawg
anuches
frodo5
alvinhs
jjj88
samleon5
calke
chichi5000
supermelo
leroy7
psycharchitect2
jeremiahhall
jakemauldin
bricz
artn
sancho2
omercel
markome
snakeh
eduard27
circaterp
j_sully
jasonxu
grizul
hrvatska420
juanpabl
jed5
iam66
misterton
treetown
qor
starsher
zeldre
gheis
mojococo
jefsch7
blanchella
['strickj', 'rashid0', 'bryg', 'yellowfupa', 'mhmmders', 'ksali', 'matthewdalton', 'jo2e', 'mlamarc', 'blueduck123', 'geldon12345', 'stephen47', 'ro

In [34]:
players

['strickj',
 'rashid0',
 'bryg',
 'yellowfupa',
 'mhmmders',
 'ksali',
 'matthewdalton',
 'jo2e',
 'mlamarc',
 'blueduck123',
 'geldon12345',
 'stephen47',
 'rob311',
 'beegal',
 'student808',
 'jorgeaa',
 'kev_98',
 'topat',
 'q7q',
 'eyezek',
 'mhyderali',
 'dbotts5',
 'raximo',
 'aceshi',
 'jimmyro',
 'djd1234',
 'shambhavisri',
 'noor197',
 'sergenian',
 'rod12',
 'dustydust',
 'testud',
 'masterdron',
 'islandof',
 'kantola',
 'excalibir',
 'pidler',
 'johnlong33',
 'krabd',
 'fastenf',
 'badger6',
 'kingjiv',
 'jerronisback',
 'bigshot81',
 'sivasagar',
 'magdy22',
 'zeban',
 'hash777',
 'mn1chess',
 'kelas111',
 'jeanrus',
 'piacentifrank',
 'mrmrs',
 'jharris1',
 'the_chosen_one10',
 'frozenfang',
 'glerps',
 'sd101',
 'trainrec',
 'bagobo',
 'des17',
 'omlu',
 'crnrfght',
 'chrondawg',
 'anuches',
 'frodo5',
 'alvinhs',
 'jjj88',
 'samleon5',
 'calke',
 'chichi5000',
 'supermelo',
 'leroy7',
 'psycharchitect2',
 'jeremiahhall',
 'jakemauldin',
 'bricz',
 'artn',
 'sancho2',
 '